In [50]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pandas_datareader as pdr
import matplotlib as plt
import openpyxl as opl
import statistics as st
import scipy as sp
import re
import glob
from datetime import datetime

# Problem 1

## Data Cleaning

In [12]:
#gets files from path
#assumes you have all the right files in 'data/' path already
#some of the csvs from French website require cleaning in excel
path = r'Problem1/data'
filenames = glob.glob(path + "/*.csv")

#imports files from FRED data into dic, indexing by date
data = {}
for filename in filenames:
    file = pd.read_csv(filename,index_col='DATE',infer_datetime_format=True)
    file.index = pd.to_datetime(file.index,format="%Y%m")
    data[filename] = file

In [15]:
portfolio = data['Problem1/data/25_Portfolios_5x5.csv']
risk_free = data['Problem1/data/RF_Data.csv']

# Part a)

### Compute Averge Returns for Each Portfolio and Table with Returns

In [19]:
#takes a 1x25 dataframe as input and outputs a 5x5 grid of values
def create_table(values_df,columns=['LoBM', 'BM2', 'BM3', 'BM4', 'HiBM'],indices = ['SMALL', 'ME2', 'ME3', 'ME4', 'BIG']):
    table = pd.DataFrame(columns=columns)
    for i in range(0,len(values_df),5):
        row = pd.DataFrame(values_df.iloc[i:i+5].T.values,columns=columns)
        table = table.append(row)

    table.index = indices
    return(table)

In [28]:
#creates return table from sum table
avg_returns = pd.DataFrame(data=round(portfolio.mean(axis = 0),2),columns=['avg'])
avg_returns_table = create_table(avg_returns)
avg_returns_table

,LoBM,BM2,BM3,BM4,HiBM
SMALL,0.76,1.24,1.20,1.39,1.51
ME2,0.97,1.21,1.26,1.28,1.41
ME3,0.97,1.20,1.14,1.27,1.38
ME4,1.07,1.05,1.10,1.22,1.25
BIG,0.96,0.94,0.97,0.89,1.05


# Part b)

### COMPUTE Average MONTHLY BETA RETURNS AND CREATE TABLE

In [21]:
mkt_return = pd.DataFrame(data=portfolio.mean(axis = 1),columns=['avg'])

In [22]:
beta = []
for i in portfolio.columns:
    cov_returns_i = np.cov(portfolio[i], mkt_return['avg'])
    cov_returns_i = pd.DataFrame(cov_returns_i)
    beta_i = round(cov_returns_i.iloc[0,1]/cov_returns_i.iloc[1,1],2)
    beta.append(beta_i)

beta = pd.DataFrame(beta)

In [23]:
#creates beta table from sum table
beta_table = create_table(beta)
beta_table

,LoBM,BM2,BM3,BM4,HiBM
SMALL,1.36,1.21,1.10,1.05,1.10
ME2,1.27,1.12,1.03,0.99,1.14
ME3,1.16,1.03,0.93,0.93,1.05
ME4,1.02,0.94,0.90,0.90,1.01
BIG,0.73,0.74,0.69,0.75,0.86


### COMPUTE PORTFOLIOS STDEV

In [24]:
#computes stdev
stdev = []
for i in portfolio.columns:
    stdev_returns_i = round(np.sqrt(st.variance(portfolio[i])),2)
    stdev.append(stdev_returns_i)

stdev = pd.DataFrame(stdev)

In [25]:
#creates stdev table from sum table
stdev_table = create_table(stdev)
stdev_table

,LoBM,BM2,BM3,BM4,HiBM
SMALL,7.96,6.97,6.05,5.84,6.30
ME2,7.16,6.04,5.52,5.36,6.22
ME3,6.56,5.51,5.03,5.10,5.86
ME4,5.83,5.13,4.97,5.04,5.81
BIG,4.62,4.40,4.31,4.75,5.64


### COMPUTE PORTFOLIOS SHARPE RATIOS

In [30]:
#computes sharpe ratios
avg_risk_free = risk_free.mean()
sharpe_ratio = []
for i in range(0,5):
    for j in range(0,5):
        sharpe_ratio_ij = round((avg_returns_table.iloc[i,j] - avg_risk_free)/stdev_table.iloc[i,j],2)
        sharpe_ratio.append(sharpe_ratio_ij)
    
sharpe_ratio = pd.DataFrame(sharpe_ratio)

In [33]:
#creates sharpe ratio table from sum table
sharpe_ratio_table = create_table(sharpe_ratio)
sharpe_ratio_table

,LoBM,BM2,BM3,BM4,HiBM
SMALL,0.05,0.12,0.14,0.17,0.18
ME2,0.08,0.14,0.16,0.17,0.17
ME3,0.09,0.15,0.15,0.18,0.17
ME4,0.12,0.13,0.15,0.17,0.15
BIG,0.13,0.13,0.14,0.11,0.12


## Part c)

### COMPUTE T-STATISTICS FOR EACH PORTFOLIO

In [34]:
true_mean = 0
t_statistic = []
for i in range(0,5):
    for j in range(0,5):
        t_ij = round((avg_returns_table.iloc[i,j] - true_mean)/stdev_table.iloc[i,j],2)
        t_statistic.append(t_ij)

t_statistic = pd.DataFrame(t_statistic)

In [35]:
#creates return table from sum table
t_stat_table = create_table(t_statistic)
t_stat_table

,LoBM,BM2,BM3,BM4,HiBM
SMALL,0.10,0.18,0.20,0.24,0.24
ME2,0.14,0.20,0.23,0.24,0.23
ME3,0.15,0.22,0.23,0.25,0.24
ME4,0.18,0.20,0.22,0.24,0.22
BIG,0.21,0.21,0.23,0.19,0.19


In [36]:
##I'm not sure what its asking for here, when it say's 5th quntile minus 1st quintile, is it asking for all of the
##biggest size firms with any book to market minus all of the low size firms with any book to market
##or is it asking for only biggest book to market and biggest size firms minus the smallest B-M and smallest size firms

In [37]:
big = list(portfolio.columns[-5:])
small = list(portfolio.columns[:5])
np.mean(portfolio[big].mean(axis=1)-portfolio[small].mean(axis=1))
# portfolio["HiBM-LoBM"] = portfolio[big]-portfolio[small]
# stdev_table

-0.25764140804597707

In [38]:
avg_return_table

,LoBM,BM2,BM3,BM4,HiBM
SMALL,0.76,1.24,1.20,1.39,1.51
ME2,0.97,1.21,1.26,1.28,1.41
ME3,0.97,1.20,1.14,1.27,1.38
ME4,1.07,1.05,1.10,1.22,1.25
BIG,0.96,0.94,0.97,0.89,1.05


In [39]:
avg_return_table['HiBM'].mean() - avg_return_table['LoBM'].mean()

0.3739999999999998

In [40]:
q5_minus_q1 = portfolio['BIG HiBM'] - portfolio['SMALL LoBM']
q5_minus_q1.mean()
# avg_q5_minus_q1 = np.mean(q5_minus_q1)
# stdev_q5_minus_q1 = np.sqrt(st.variance(q5_minus_q1))

0.2884751436781609

# Problem 2

In [66]:
#gets files from path
#assumes you have all the right files in 'data/' path already
#some of the csvs from French website require cleaning in excel
path = r'Problem2/data'
filenames = glob.glob(path + "/*.csv")

#imports files from FRED data into dic, indexing by date
data = {}
for filename in filenames:
    print(filename)
    file = pd.read_csv(filename,index_col='DATE',infer_datetime_format=True)
    file.index = pd.to_datetime(file.index,format="%Y%m")
    data[filename] = file

Problem2/data/F-F_Research_Data_5_Factors_2x3.csv
Problem2/data/F-F_Momentum_Factor-2.csv
Problem2/data/F-F_ST_Reversal_Factor.csv
Problem2/data/F-F_Research_Data_Factors-2.csv


In [67]:
Factors_2x3 = data['Problem2/data/F-F_Research_Data_5_Factors_2x3.csv']
momentum = data['Problem2/data/F-F_Momentum_Factor-2.csv']
reversal = data['Problem2/data/F-F_ST_Reversal_Factor.csv']
Factors_2 = data['Problem2/data/F-F_Research_Data_Factors-2.csv']

## Part a)